In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
import seaborn as sns

In [ ]:
df=pd.read_csv("/kaggle/input/talking-stage-training-data/training_data.csv")

In [ ]:
df.nunique()

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download("stopwords")
nltk.download("punkt")

In [ ]:
def text_preprocessing(text):
    data=text.lower()
    data_re=re.sub(r'\W+'," ",data)
    data_tokenize=word_tokenize(data_re)
    tokens=[word for word in data_tokenize if word not in stopwords.words('english')]
    return " ".join(tokens)


In [ ]:
df["text"]=df["Text"].apply(text_preprocessing)

In [ ]:
df

In [ ]:
tfidf=TfidfVectorizer()
da=tfidf.fit_transform(df["text"])
da=da.toarray()
da=pd.DataFrame(da,columns=tfidf.get_feature_names_out())

In [ ]:
da




In [ ]:
one_hot=OneHotEncoder(sparse_output=False)
one=one_hot.fit_transform(df[["Label"]])
one=pd.DataFrame(one,columns=one_hot.get_feature_names_out())

In [ ]:
len(one.nunique())

# # Deep learning

In [ ]:
from keras.layers import Dense,Input,Dropout,TimeDistributed,BatchNormalization
from keras.models import Model
from keras.initializers import HeNormal
from keras.regularizers import l2
from keras.callbacks import EarlyStopping

In [ ]:
da.shape

In [ ]:
inputs=Input(shape=(1351,))
d=Dense(32,activation="relu",kernel_regularizer=l2(0.01),kernel_initializer=HeNormal())(inputs)
d=Dense(3200,activation="relu",kernel_regularizer=l2(0.01),kernel_initializer=HeNormal())(d)
outputs=Dense(39,activation="softmax")(d)
model=Model(inputs,outputs)
model.compile(optimizer='adam',loss="categorical_crossentropy",metrics=["accuracy"])

In [65]:
model.fit(da,one,batch_size=32,epochs=10,validation_split=0.2)

Epoch 1/10
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9682 - loss: 1.2746 - val_accuracy: 0.0780 - val_loss: 9.2655
Epoch 2/10
89/89 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9443 - loss: 1.2772 - val_accuracy: 0.0780 - val_loss: 9.1889
Epoch 3/10
89/89 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9664 - loss: 1.2092 - val_accuracy: 0.0752 - val_loss: 9.3642
Epoch 4/10
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9643 - loss: 1.1675 - val_accuracy: 0.0752 - val_loss: 9.4202
Epoch 5/10
89/89 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9685 - loss: 1.1231 - val_accuracy: 0.0738 - val_loss: 9.4723
Epoch 6/10
89/89 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9547 - loss: 1.1372 - val_accuracy: 0.0752 - val_loss: 9.4425
Epoch 7/10
89/89 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9625 - loss: 1.0905 - val_accuracy: 0.0709 - val_loss: 9.5274
Epoch 8/10
89/89 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9636 - loss: 1.0535 - val_accuracy: 0.0780 - val_loss:

In [66]:
y_pred=model.predict(da.iloc[0:100])

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


In [67]:
loss, accuracy = model.evaluate(da.iloc[10:100], one.iloc[10:100])


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9655 - loss: 0.9094 


In [ ]:
sns.lineplot(da.iloc[0:100])
sns.lineplot(y_pred)
plt.legend().set_visiblity(False)

/opt/conda/lib/python3.10/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
/opt/conda/lib/python3.10/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):


In [ ]:
sns.violinplot(da.iloc[0:100])
sns.violinplot(y_pred,color="red")

In [ ]:
sns.histplot(da.iloc[0:100],kde=True)
sns.histplot(y_pred,kde=True,color="red")